### Generate Wiki Text 

##### Download data from https://metamind.io/research/the-wikitext-long-term-dependency-language-modeling-dataset/

In [18]:
#mini-demo
from urllib.request import urlretrieve
import os 
from os.path import isfile, isdir
import zipfile 
from tqdm import tqdm
import numpy as np #vectorization
import random #generate probability distribution 
import tensorflow as tf #ml
import datetime #clock training time

### First download data 

In [45]:
#### process bar
class DLProgress(tqdm):
    last_block = 0
    def hook(self, block_num=1, block_size=1, total_size=None):
        self.total = total_size
        self.update((block_num - self.last_block) * block_size)
        self.last_block = block_num

## download file 
data_path = './wikitext'
if isdir(data_path):
    print('Data already exist')
else:
    if not isdir(data_path):
        os.mkdir(data_path)
    zip_file = os.path.join(data_path,'wikitext-103-v1.zip')
    with DLProgress(unit='B', unit_scale=True, miniters=1, desc='wikidata') as pbar:
        urlretrieve('https://s3.amazonaws.com/research.metamind.io/wikitext/wikitext-103-v1.zip',
                    zip_file,
                    pbar.hook)
    with zipfile.ZipFile(os.path.join(data_path,'wikitext-103-v1.zip')) as myzip:
        myzip.extractall(data_path)
    ## remove zip file 
    os.remove(data_path+'/wikitext-103-v1.zip')

data_file_path = "./wikitext/wikitext-103"
train_file = os.path.join(data_file_path,'wiki.train.tokens')
validate_file = os.path.join(data_file_path,'wiki.valid.tokens')

Data already exist


### Read data

In [3]:
#lets open the text
#native python file read function
text = open('wikitext/wiki.train.tokens').read()
print('text length in number of characters:', len(text))
print('head of text:')
print(text[:1000]) #all tokenized words, stored in a list called text

text length in number of characters: 538360726
head of text:
 
 = Valkyria Chronicles III = 
 
 Senjō no Valkyria 3 : <unk> Chronicles ( Japanese : 戦場のヴァルキュリア3 , lit . Valkyria of the Battlefield 3 ) , commonly referred to as Valkyria Chronicles III outside Japan , is a tactical role @-@ playing video game developed by Sega and Media.Vision for the PlayStation Portable . Released in January 2011 in Japan , it is the third game in the Valkyria series . Employing the same fusion of tactical and real @-@ time gameplay as its predecessors , the story runs parallel to the first game and follows the " Nameless " , a penal military unit serving the nation of Gallia during the Second Europan War who perform secret black operations and are pitted against the Imperial unit " <unk> Raven " . 
 The game began development in 2010 , carrying over a large portion of the work done on Valkyria Chronicles II . While it retained the standard features of the series , it also underwent multiple adjustments

In [7]:
## get the set of characters and sort them 
chars = sorted(list(set(text)))
char_size = len(chars)
print('number of characters:', char_size)
print(chars[:20])

number of characters: 1250
['\n', ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2']


In [16]:
## chrate char to id and id to char map 
char2id = {c:i for i,c in enumerate(chars)}
id2char = {i:c for i,c in enumerate(chars)}

In [ ]:
## generate probability of each next charater 
def sample(prediction):
    r = random.uniform(0,1)
    s = 0 
    char_id = len(prediction)-1
    #for each char prediction probability 
    for i in range(len(prediction)):
        s+= prediction[i]
        if s >= r:
            char_id = i 
            break 
    
    char_one_hot = np.zeros(shape[char_size])
    char_one_hot[char_id] = 1.0
    return char_one_hot